In [101]:
import pandas as pd

In [102]:
from aicentro.session import Session
session = Session(verify=False)

In [103]:
import tensorflow as tf
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.optimizers import RMSprop

In [104]:
voc_df = pd.read_csv('/aihub/workspace/AIProject_202007/Data/voc_preScale_final_2_20200721.csv', encoding = 'UTF-8')

In [105]:
del voc_df['Unnamed: 0']

In [106]:
voc_df

,상담대분류,상담중분류,상담소분류,상담일자,상품구분,급지,해지일,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
0,4,10,27,2016-01-01,0,3,2016-06-15,25,86,68,-1.0,166,False
1,4,10,27,2016-01-01,0,4,2015-12-31,25,86,68,-1.0,-1,False
2,4,10,27,2016-01-01,0,2,1999-01-01,25,86,68,-1.0,-6209,False
3,2,11,56,2016-01-01,1,1,1999-01-01,18,75,55,16.0,-6209,False
4,4,10,27,2016-01-02,0,3,1999-01-01,25,86,68,-1.0,-6210,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40483,2,18,53,2020-07-14,1,4,1999-01-01,25,86,68,-1.0,-7865,False
40484,2,18,40,2020-07-14,1,2,1999-01-01,25,86,68,-1.0,-7865,False
40485,3,6,47,2020-07-14,1,3,1999-01-01,25,86,68,-1.0,-7865,False
40486,2,18,54,2020-07-14,1,6,1999-01-01,25,86,68,-1.0,-7865,False


In [107]:
voc_df.corr()

,상담대분류,상담중분류,상담소분류,상품구분,급지,AS신청유형,AS원인유형,AS처리유형,AS처리시간,cnslt_rscs_diff,rscs_yn
상담대분류,1.000000,-0.613951,-0.064542,-0.858035,-0.049287,0.100360,0.140734,0.132088,-0.050805,0.050805,-0.103012
상담중분류,-0.613951,1.000000,0.104928,0.309086,-0.018732,0.046618,0.075836,0.074865,-0.033488,-0.025873,0.123457
상담소분류,-0.064542,0.104928,1.000000,0.150294,-0.015071,0.088229,0.113209,0.117037,-0.069336,0.087981,0.096098
상품구분,-0.858035,0.309086,0.150294,1.000000,0.086309,-0.118083,-0.162363,-0.155013,0.053537,-0.025106,0.078850
급지,-0.049287,-0.018732,-0.015071,0.086309,1.000000,0.023191,0.014625,0.009234,0.040602,-0.048225,-0.046930
AS신청유형,0.100360,0.046618,0.088229,-0.118083,0.023191,1.000000,0.269823,0.255534,-0.082356,0.050427,0.017585
AS원인유형,0.140734,0.075836,0.113209,-0.162363,0.014625,0.269823,1.000000,0.911809,-0.137373,0.055328,0.015896
AS처리유형,0.132088,0.074865,0.117037,-0.155013,0.009234,0.255534,0.911809,1.000000,-0.126009,0.058493,0.019658
AS처리시간,-0.050805,-0.033488,-0.069336,0.053537,0.040602,-0.082356,-0.137373,-0.126009,1.000000,-0.022638,-0.016578
cnslt_rscs_diff,0.050805,-0.025873,0.087981,-0.025106,-0.048225,0.050427,0.055328,0.058493,-0.022638,1.000000,0.433840


In [108]:
val_columns = ['상담대분류', '상담중분류', '상담소분류', '급지', '상품구분', 'AS신청유형', 'AS원인유형', 'AS처리유형', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [109]:
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,4,10,27,3,0,25,86,68,False
1,4,10,27,4,0,25,86,68,False
2,4,10,27,2,0,25,86,68,False
3,2,11,56,1,1,18,75,55,False
4,4,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,2,18,53,4,1,25,86,68,False
40484,2,18,40,2,1,25,86,68,False
40485,3,6,47,3,1,25,86,68,False
40486,2,18,54,6,1,25,86,68,False


In [110]:
cnslt_big = [
'(인)고객불만'
,'가입해지'
,'고객불만'
,'고객불만 이관'
,'서비스 문의 및 신청'
,'품질'
,'회사 관련 문의'
]
toCharCoder_cnslt_big={i:k for i,k in enumerate(cnslt_big)}


In [120]:
l=[toCharCoder_cnslt_big[i] for i in voc_df['상담대분류']]
s=pd.Series(l)
voc_df['상담대분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,10,27,3,0,25,86,68,False
1,서비스 문의 및 신청,10,27,4,0,25,86,68,False
2,서비스 문의 및 신청,10,27,2,0,25,86,68,False
3,고객불만,11,56,1,1,18,75,55,False
4,서비스 문의 및 신청,10,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,18,53,4,1,25,86,68,False
40484,고객불만,18,40,2,1,25,86,68,False
40485,고객불만 이관,6,47,3,1,25,86,68,False
40486,고객불만,18,54,6,1,25,86,68,False


In [100]:
cnslt_mid = [
'AS처리점 유발'
,'CRG호전환'
,'고객센터/유통망관련문의'
,'과기정통부'
,'기타 단체'
,'내방고객'
,'대한주부클럽연합회'
,'방송통신위원회'
,'본사 전화민원'
,'서비스 문의 및 신청'
,'서비스 신청'
,'설치(이전)/AS'
,'소비자보호원'
,'소비자신문'
,'소비자연맹'
,'수신기'
,'수신장애'
,'시민의모임'
,'영업'
]

toCharCoder_cnslt_mid={i:k for i,k in enumerate(cnslt_mid)}

In [121]:
l=[toCharCoder_cnslt_mid[i] for i in voc_df['상담중분류']]
s=pd.Series(l)
voc_df['상담중분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,27,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,27,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,56,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,27,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,53,4,1,25,86,68,False
40484,고객불만,영업,40,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,47,3,1,25,86,68,False
40486,고객불만,영업,54,6,1,25,86,68,False


In [123]:
cnslt_small = [
'(AS기사) 방문약속 미이행'
,'(AS기사) 방문약속 미이행'
,'(AS기사) 불친절'
,'(AS기사) 처리지연'
,'(AS기사) 후처리 미흡'
,'(문의) 시청불가'
,'(문의) 신호미약'
,'(설치기사) 방문약속 미이행'
,'(설치기사) 불친절'
,'(설치기사) 처리지연'
,'(설치기사) 후처리 미흡'
,'(접수) 기타메시지'
,'(접수) 리모콘 동작불가'
,'(접수) 수신기 동작불가'
,'(접수) 수신설비 파손 분실'
,'(접수) 신호미약'
,'(접수) 영상불량'
,'(접수) 음성불량'
,'(접수)스마트카드 오류 메시지'
,'AS 중복발생'
,'AS비용불만(즉납)'
,'AS점 직원 불친절'
,'AS점불만'
,'AS지연'
,'AS출동지연(도서지역)'
,'AS출동지연(약속미이행)'
,'AS출동지연(중복)SMS'
,'SkyChoice'
,'가격정책(제도)'
,'가전결합 불만'
,'개인정보 침해'
,'고객재산 파손 및 손괴'
,'기사칭찬'
,'기업 상품'
,'기타 단체'
,'기타(특이사례)'
,'명의도용'
,'방송품질'
,'사은품 미지금(약속미준수)'
,'상담'
,'상품/약정안내 미흡'
,'설치(이전)/AS'
,'설치점 불만'
,'설치점불만'
,'설치지연'
,'쌍방향 서비스'
,'업셀링 요금/약정 안내 미흡'
,'영업'
,'영업점불만'
,'영업점불만(신청서 대필)'
,'영업점불만(오처리/처리누락)'
,'영업점불만(임의개통_가개통)'
,'영업점불만(임의개통_명의도용)'
,'영업점불만(임의개통_이중가입)'
,'영업점불만(임의개통_일방적가입)'
,'이용불편'
,'잦은 AS'
,'청구/환불'
,'콘텐츠'
,'클레임처리지연'
,'패키지 체계 및 요금'
,'패키지 추가'
,'해지'
,'해지 접수'
]

toCharCoder_cnslt_small={i:k for i,k in enumerate(cnslt_small)}

In [124]:
l=[toCharCoder_cnslt_small[i] for i in voc_df['상담소분류']]
s=pd.Series(l)
voc_df['상담소분류'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,0,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,0,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,1,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,0,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,1,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,1,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,1,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,1,25,86,68,False


In [125]:
prdt_type = [
'없음'
,'위성'
,'인터넷'
]

toCharCoder_prdt_type={i:k for i,k in enumerate(prdt_type)}

In [126]:
l=[toCharCoder_prdt_type[i] for i in voc_df['상품구분']]
s=pd.Series(l)
voc_df['상품구분'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,4,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,2,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,1,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,3,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),4,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,2,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,3,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),6,위성,25,86,68,False


In [129]:
region_lvl = [
'1등급'
,'2등급'
,'3등급'
,'4등급'
,'5등급'
,'6등급'
,'7등급'
,'8등급'
,'급지없음'
]

toCharCoder_region_lvl={i:k for i,k in enumerate(region_lvl)}

In [131]:
l=[toCharCoder_region_lvl[i] for i in voc_df['급지']]
s=pd.Series(l)
voc_df['급지'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,25,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,25,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,25,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,18,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,25,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,25,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,25,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,25,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,25,86,68,False


In [132]:
as_apply = [
'B/S점검'
,'IP백업고객 B/S점검'
,'IP연결요청'
,'PPV 오류 메시지'
,'PVR 녹화/시청 불가'
,'TQC'
,'USB 플레이어/타임머신 동작 불가'
,'VOD 시청 불가'
,'Wifi 사용불가'
,'Wifi 연결안됨'
,'[휴일]수신기 전원 미작동'
,'기타'
,'기타'
,'기타 메시지'
,'네트워크 연결확인 메시지'
,'리모콘 동작불가'
,'리모콘 사용법 모름'
,'모뎀 전원 미동작'
,'수신기 동작불가'
,'수신기 업그레이드안됨'
,'수신기 전원 미동작'
,'수신설비 파손 분실'
,'스마트카드 오류 메시지'
,'신호미약 메시지'
,'실버리모콘'
,'없음'
,'영상 불량'
,'영상 안아놈'
,'음성 불량'
,'음성 안나옴'
,'인터넷 사용불가'
,'재해지역AS발생'
,'케이블 연결방법 모름'
]

toCharCoder_as_apply={i:k for i,k in enumerate(as_apply)}

In [133]:
l=[toCharCoder_as_apply[i] for i in voc_df['AS신청유형']]
s=pd.Series(l)
voc_df['AS신청유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,86,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,없음,86,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,없음,86,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,수신기 동작불가,75,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,86,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,없음,86,68,False
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,없음,86,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,없음,86,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,없음,86,68,False


In [134]:
as_cause = [
'A/S취소'
,'A/V케이블 분리/단선'
,'A/V케이블 불량'
,'A/V케이블 오연결'
,'AS관련 클레임'
,'B/S점검'
,'IP연결요청'
,'LNB 동작불가'
,'LNB 파손'
,'LNB 파손/분실.동작불가'
,'LNB불량'
,'LTE모뎀 동작불가'
,'TP유니트 불량'
,'UTP 케이블 분리/단선'
,'UTP커넥터불량'
,'[FTTH] 포트불량'
,'[FTTH] 환경설정오류'
,'[IP공유기] 고객부주의(사용미숙)'
,'[IP공유기] 접속불량'
,'[IP공유기] 제품불량(제조사)'
,'[NE] 네트워크장비장애'
,'[Ntopia] 불복구'
,'[Ntopia] 포트불량'
,'[TOK] 비고장'
,'[고객취소] 비고장'
,'[구내단자함] 접속불량'
,'[기술상담] 문의성'
,'[기타] 문의성'
,'[기타] 비고장'
,'[모뎀] 고객부주의(사용미숙)'
,'[모뎀] 분실'
,'[모뎀] 불복구'
,'[모뎀] 속도저하'
,'[모뎀] 장애/공사/피해'
,'[모뎀] 접속불량'
,'[모뎀] 제품불량(제조사)'
,'[모뎀] 환경설정오류'
,'[세대단자함] 공법불량'
,'[세대단자함] 접속불량'
,'[실내선] 고객부주의(사용미숙)'
,'[실내선] 장애/공사/피해'
,'[실내선] 접속불량'
,'[인입광] 고객요청(민원)'
,'[인입광] 공법불량'
,'[인입광] 시설노후'
,'[인입광] 장애/공사/피해'
,'[인입광] 접속불량'
,'[인입광] 고객부주의(사용미숙)'
,'[인입선] 시설노후'
,'[인입선] 절연불량'
,'[인입선] 접속불량'
,'[자연회복] 자연회복'
,'[콘넥터] 접속불량'
,'[폐문] 비고장'
,'가스배관 등 위험지역 설치'
,'공유기 사용 미숙지'
,'공유기/허브 동작 불가'
,'기타'
,'녹화방법 미숙지'
,'도시가스배관정비'
,'동작 불가'
,'동작불가'
,'동툭케이블 분리/단선'
,'동축콘넥터 접불'
,'리모콘 동작불가'
,'리모콘 분실'
,'리모콘 추가'
,'리모콘 파손'
,'리모콘사용법 미숙지'
,'방수처리 불량'
,'불량'
,'설치관련 클레임'
,'수신기 LNB전원공급 불가'
,'수신기 동작불가'
,'수신기 분실'
,'수신기 전원 동작불가'
,'수신기 파손'
,'수신기 파손/분식/동작불가'
,'스마트카드 동작불가'
,'스마트카드 분실'
,'신호 불량'
,'안테나 고장불량(앙카볼트 미사용 등)'
,'안테나 동갖불가'
,'안테나 방향 틀어짐'
,'안테나 방향 틀어짐(고객사유)'
,'안테나 파손/분실'
,'없음'
,'일시적 장애'
,'잭 접불'
,'저잡음증폭기 불량'
,'전산 작업 오류'
,'전원'
,'회선 분리/단선'
,'접속불량'
,'정상확인'
,'지상파HD채널설정방법 미숙지'
,'자상파전환방법 미숙지'
,'취약지역 안테나 설치'
,'케이블연결방법 미숙지'
,'타 가입자 안테나 분배 설치'
,'파손/분실/동작불가'
,'해상도 변경 미숙지'
]

toCharCoder_as_cause={i:k for i,k in enumerate(as_cause)}

In [135]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS원인유형']]
s=pd.Series(l)
voc_df['AS원인유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,68,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,없음,없음,68,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,없음,없음,68,False
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,수신기 동작불가,수신기 전원 동작불가,55,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,68,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,없음,없음,68,False
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,없음,없음,68,False
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,없음,없음,68,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,없음,없음,68,False


In [136]:
as_process = [
'(외부자산)시설교체 및 수리 안내'
,'A/S취소'
,'A/V케이블 교체'
,'A/V케이블 재연결'
,'AS관련 클레임 처리'
,'AS취소'
,'IP연결요청'
,'LNB교체'
,'LNB교체'
,'LTE모뎀 교체'
,'Modulator 교체'
,'UTP 케이블 포설'
,'UTP커넥터 접속'
,'[FTTH] 포트교체'
,'[FTTH] 프로파일재설정'
,'[IP공유기] 교체(모뎀/ONT/STB)'
,'[IP공유기] 수리의뢰(고객)'
,'[IP공유기] 환경재설정'
,'[NE] 무출동네트워크장비복구'
,'[Ntopia] 포트교체'
,'[Ntopia] 프로파일재설정'
,'[구내단자함] 재접속'
,'[모뎀] 교체(모뎀/ONT/STB)'
,'[모뎀] 불복구제거(RESET)'
,'[모뎀] 재접속'
,'[모뎀] 환경재설정'
,'[세대단자함] 선로재구성(인입선 등)'
,'[세대단자함] 재접속'
,'[실내선]LAB케이블 교체'
,'[실내선]선로재구성(인입선 등)'
,'[실내선]재접속'
,'[인입광] 광콘넥터재접속'
,'[인입광] 교체(모뎀/ONT/STB)'
,'[인입광] 선로재구성(인입성 등)'
,'[인입광] 회선정비'
,'[인입선] 선로재구성(인입선 등)'
,'[인입선] 재접속'
,'[콘넥터] 광콘넥터재접속'
,'[콘넥터] 재접속'
,'공유기 사용안내'
,'공유기/허브 전원 재연결'
,'교체'
,'교체-IF증폭기-기타(미승인)'
,'교체-IF증폭기-아진'
,'교체-IF증폭기-청화'
,'교체/수리 안내'
,'기 타'
,'기술상담'
,'기타'
,'녹화방법안내'
,'동축케이블 포설'
,'동축콘넥터 접속'
,'리모콘 교체'
,'리모콘 추가'
,'리모콘사용법 안내'
,'무상수리교체(무료보증기간)'
,'방수(실리콘)처리'
,'배선 등 현장점검'
,'설치관련 클레임 처리'
,'수신기 교체'
,'수신기 전원 재연결'
,'스마트카드 교체'
,'안테나 교체'
,'안테나 교체/설치'
,'안테나 방향 조정'
,'안테나 재설치 또는 점검'
,'안테나재설치'
,'잉카볼트사용 및 고정상태'
,'없음'
,'유니트 교체'
,'유상구매교체(수리불가)'
,'유상수리교체(수리후재설치)'
,'자가보유교체(고객보유분)'
,'자동복구'
,'자연회복'
,'저잡음증폭기교체'
,'전산 작업'
,'전산작업'
,'전원 재입전'
,'전화선 연결'
,'정상확인/종료'
,'조치없음'
,'증폭기 신호조정'
,'증폭기전원재연결'
,'증폭기 추가(지사통보)'
,'지상파HD채널설정방법 안내'
,'지상파 전환방법 안내'
,'커넥터 재작업(재접속)'
,'케이블 연결방법 안내'
,'케이블 재포설(교체)'
,'타센터 AS처리'
,'해상도 변경 안내'
,'현장수리완료(OTA전송)']

toCharCoder_as_process={i:k for i,k in enumerate(as_process)}

In [137]:
l=[toCharCoder_as_cause[i] for i in voc_df['AS처리유형']]
s=pd.Series(l)
voc_df['AS처리유형'] = s
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,리모콘사용법 미숙지,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,없음,없음,리모콘사용법 미숙지,False
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,없음,없음,리모콘사용법 미숙지,False
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,수신기 동작불가,수신기 전원 동작불가,공유기 사용 미숙지,False
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,리모콘사용법 미숙지,False
...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,없음,없음,리모콘사용법 미숙지,False
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,없음,없음,리모콘사용법 미숙지,False
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,없음,없음,리모콘사용법 미숙지,False


In [140]:
# # 데이터를 읽고 순서를 섞는다
voc_model_df=voc_df.sample(frac=1).reset_index(drop=True)
voc_model_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn
0,고객불만,설치(이전)/AS,(설치기사) 후처리 미흡,3등급,위성,없음,없음,리모콘사용법 미숙지,False
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,없음,없음,리모콘사용법 미숙지,False
2,고객불만 이관,과기정통부,청구/환불,1등급,위성,없음,없음,리모콘사용법 미숙지,False
3,서비스 문의 및 신청,서비스 신청,SkyChoice,2등급,없음,없음,없음,리모콘사용법 미숙지,False
4,고객불만 이관,소비자연맹,청구/환불,5등급,위성,케이블 연결방법 모름,케이블연결방법 미숙지,잭 접불,False
...,...,...,...,...,...,...,...,...,...
40483,서비스 문의 및 신청,서비스 신청,SkyChoice,7등급,없음,없음,없음,리모콘사용법 미숙지,False
40484,고객불만,영업,영업점불만,5등급,위성,없음,없음,리모콘사용법 미숙지,False
40485,고객불만,영업,영업점불만,2등급,위성,없음,없음,리모콘사용법 미숙지,True
40486,서비스 문의 및 신청,서비스 신청,패키지 추가,2등급,없음,없음,없음,리모콘사용법 미숙지,False


In [139]:
voc_model_df.dtypes

상담대분류      object
상담중분류      object
상담소분류      object
급지         object
상품구분       object
AS신청유형     object
AS원인유형     object
AS처리유형     object
rscs_yn      bool
dtype: object

In [141]:
!pip install JPype1
!pip install konlpy

Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.
Looking in indexes: http://aicentro-nexus-svc:8081/hub-nexus/repository/pypi-group/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [142]:
from konlpy.tag import Kkma
kkma = Kkma()

In [145]:
cnslt_dic = print(kkma.nouns(u'고객불만 이관'))

['고객', '고객불만', '불만', '이관']


In [148]:
voc_df["상담"] = voc_df["상담대분류"] + voc_df["상담중분류"] + voc_df["상담소분류"]

In [149]:
voc_df['AS'] = voc_df["AS신청유형"] + voc_df['AS원인유형'] + ['AS처리유형']

In [150]:
voc_df

,상담대분류,상담중분류,상담소분류,급지,상품구분,AS신청유형,AS원인유형,AS처리유형,rscs_yn,상담,AS
0,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,리모콘사용법 미숙지,False,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형
1,서비스 문의 및 신청,서비스 신청,SkyChoice,5등급,없음,없음,없음,리모콘사용법 미숙지,False,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형
2,서비스 문의 및 신청,서비스 신청,SkyChoice,3등급,없음,없음,없음,리모콘사용법 미숙지,False,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형
3,고객불만,설치(이전)/AS,잦은 AS,2등급,위성,수신기 동작불가,수신기 전원 동작불가,공유기 사용 미숙지,False,고객불만설치(이전)/AS잦은 AS,수신기 동작불가수신기 전원 동작불가AS처리유형
4,서비스 문의 및 신청,서비스 신청,SkyChoice,4등급,없음,없음,없음,리모콘사용법 미숙지,False,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형
...,...,...,...,...,...,...,...,...,...,...,...
40483,고객불만,영업,영업점불만(임의개통_이중가입),5등급,위성,없음,없음,리모콘사용법 미숙지,False,고객불만영업영업점불만(임의개통_이중가입),없음없음AS처리유형
40484,고객불만,영업,상품/약정안내 미흡,3등급,위성,없음,없음,리모콘사용법 미숙지,False,고객불만영업상품/약정안내 미흡,없음없음AS처리유형
40485,고객불만 이관,대한주부클럽연합회,영업,4등급,위성,없음,없음,리모콘사용법 미숙지,False,고객불만 이관대한주부클럽연합회영업,없음없음AS처리유형
40486,고객불만,영업,영업점불만(임의개통_일방적가입),7등급,위성,없음,없음,리모콘사용법 미숙지,False,고객불만영업영업점불만(임의개통_일방적가입),없음없음AS처리유형


In [151]:
val_columns = ['상담', 'AS', '급지', '상품구분', 'rscs_yn']
voc_df = pd.DataFrame(voc_df, columns=val_columns)

In [152]:
voc_df

,상담,AS,급지,상품구분,rscs_yn
0,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,4등급,없음,False
1,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,5등급,없음,False
2,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,3등급,없음,False
3,고객불만설치(이전)/AS잦은 AS,수신기 동작불가수신기 전원 동작불가AS처리유형,2등급,위성,False
4,서비스 문의 및 신청서비스 신청SkyChoice,없음없음AS처리유형,4등급,없음,False
...,...,...,...,...,...
40483,고객불만영업영업점불만(임의개통_이중가입),없음없음AS처리유형,5등급,위성,False
40484,고객불만영업상품/약정안내 미흡,없음없음AS처리유형,3등급,위성,False
40485,고객불만 이관대한주부클럽연합회영업,없음없음AS처리유형,4등급,위성,False
40486,고객불만영업영업점불만(임의개통_일방적가입),없음없음AS처리유형,7등급,위성,False


In [ ]:
nlp_dic = []
ll = [kkma.nouns(i) for i in voc_df['상담']]

In [162]:
nlp_dic = kkma.nouns(u'고객수신기 동작불가수신기 전원 동작불가AS처리유형')